<a href="https://colab.research.google.com/github/GaoangLiu/AA_ipynb/blob/master/Bag_of_Words_Meets_Bags_of_Popcorn_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!curl -o train.tsv ali.140714.xyz:8000/popcorn.tsv
!curl -o test.tsv ali.140714.xyz:8000/popcorn_test.tsv
# !curl -o master.csv ali.140714.xyz:8000/imdb_master.csv
!curl -o sample.csv ali.140714.xyz:8000/popcorn_sample.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32.0M  100 32.0M    0     0  1266k      0  0:00:25  0:00:25 --:--:-- 1046k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.2M  100 31.2M    0     0  1839k      0  0:00:17  0:00:17 --:--:-- 1335k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  129M  100  129M    0     0   861k      0  0:02:33  0:02:33 --:--:--  892k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  276k  100  276k    0     0   173k      0  0:00:01  0:00:01 --:--:--  173k


In [65]:
extra = pd.read_csv('master.csv', encoding="latin-1")
extra.drop(extra[extra.label=='unsup'].index, inplace=True)
extra['sentiment'] = (extra['label'] == 'pos').astype(int)
# extra= extra[['review', 'sentiment']]
# extra.label.value_counts(), extra
extra

# train = pd.read_csv('train.tsv', sep='\t')
# train = train[['review', 'sentiment']]
# train = pd.concat([extra, train])
# train.sentiment.value_counts()

,Unnamed: 0,type,review,label,file,sentiment
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt,0
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt,0
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt,0
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt,0
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt,0
...,...,...,...,...,...,...
49995,49995,train,"Seeing as the vote average was pretty low, and...",pos,9998_9.txt,1
49996,49996,train,"The plot had some wretched, unbelievable twist...",pos,9999_8.txt,1
49997,49997,train,I am amazed at how this movie(and most others ...,pos,999_10.txt,1
49998,49998,train,A Christmas Together actually came before my t...,pos,99_8.txt,1


In [0]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 
import sklearn.metrics
import seaborn as sns

In [102]:
import os 
import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf
import tensorflow_hub as hub
import sklearn
import sklearn.metrics 
import sklearn.model_selection
import sklearn.feature_extraction
from absl import logging
import gensim.downloader as api
logging.set_verbosity(logging.INFO)

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

class OverwriteLog():
    # overwrite logging for kaggle kernel
    def info(self, msg):
        print(msg)

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token, 'v') for token in text.split(" ")]
    text = " ".join(text)
    return text

def load_data():
    train = pd.read_csv('train.tsv', sep='\t')
    test = pd.read_csv('test.tsv', sep='\t')

    # extra = pd.read_csv('master.csv', encoding="latin-1")
    # extra.drop(extra[extra.label=='unsup'].index, inplace=True)
    # extra['sentiment'] = (extra['label'] == 'pos').astype(int)
    # extra= extra[['review', 'sentiment']]

    # train = train[['review', 'sentiment']]
    # train = pd.concat([extra, train])

    train['text'] = train.review.apply(clean_text)
    train['target'] = train.sentiment
    test['text'] = test.review
    return train, test
    

def countvectorize(train, test, vocab_size, sentence_len):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size)
    text = pd.concat([train.text, test.text])
    tokenizer.fit_on_texts(text)

    sequence_train = tokenizer.texts_to_sequences(train.text)
    train_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequence_train, maxlen=sentence_len)
    logging.info('Train text tokeninzed')

    sequence_test = tokenizer.texts_to_sequences(test.text)
    test_inputs = tf.keras.preprocessing.sequence.pad_sequences(sequence_test, maxlen=sentence_len)
    logging.info('Test text tokeninzed')

    return train_inputs, test_inputs, tokenizer

def embed_word_vector(word_index, vocab_size=10000, embed_size=128, model='glove-wiki-gigaword-100'):
    glove = api.load(model) # default: wikipedia 6B tokens, uncased
    zeros = [0] * embed_size
    matrix = np.zeros((vocab_size, embed_size))
      
    for word, i in word_index.items(): 
        if i >= vocab_size or word not in glove: continue # matrix[0] is zeros, that's also why >= is here
        matrix[i] = glove[word]

    logging.info('Matrix with embedded word vector created')
    return matrix

def build_model(vocab_size = 10000, max_len = 100, embed_size = 128, embed_matrix=[]):
    text_input = tf.keras.Input(shape=(max_len, ))
    embed_text = tf.keras.layers.Embedding(vocab_size, embed_size)(text_input)
    if len(embed_matrix) > 0:
        embed_text = tf.keras.layers.Embedding(vocab_size, embed_size, \
                                        weights=[embed_matrix], trainable=False)(text_input)
        
    net = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed_text)
    net = tf.keras.layers.GlobalMaxPool1D()(net)

    net = tf.keras.layers.Dense(64, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.1)(net)

    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.1)(net)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(net)
    
    model = tf.keras.models.Model(inputs=text_input, outputs=outputs)

    return model

"""split the following codes into several chunks in Jyputer 
for clearer reading and saved variables
"""

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'split the following codes into several chunks in Jyputer \nfor clearer reading and saved variables\n'

In [103]:
vocab_size = 30_000
sentence_len = 250
embed_size = 300

train, test = load_data()
train_inputs, test_inputs, tokenizer = countvectorize(train, test, vocab_size, sentence_len)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(train_inputs, train.target, test_size=0.2, random_state=0)
# matrix = embed_word_vector(tokenizer.word_index, vocab_size, embed_size, "glove-wiki-gigaword-300")
matrix = embed_word_vector(tokenizer.word_index, vocab_size, embed_size, "fasttext-wiki-news-subwords-300")
logging.info("Data loaded and split")


INFO:absl:Train text tokeninzed
INFO:absl:Test text tokeninzed


[==================================================] 100.0% 958.5/958.4MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
INFO:absl:Matrix with embedded word vector created
INFO:absl:Data loaded and split


In [0]:
# train_inputs.shape
# tokenizer = tf.keras.preprocessing.text.Tokenizer()
# text = pd.concat([train.text, test.text])
# tokenizer.fit_on_texts(text)
# tokenizer.num_words

In [0]:
# len(tokenizer.word_index)

In [104]:
# Build model 
model = build_model(vocab_size, sentence_len, embed_size, matrix)
model.summary()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='adam', metrics=['accuracy'])
logging.info("Model built")



INFO:absl:Model built


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 250, 300)          9000000   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 250, 256)          439296    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 256)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 32)                208

In [105]:
# Run model
checkpoint = tf.keras.callbacks.ModelCheckpoint('best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

train_history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[checkpoint, earlystopping],
    batch_size=1024,
    verbose=1
)
logging.info("Model trainning complete")


Epoch 1/50
20/20 [==============================] - ETA: 0s - loss: 0.6873 - accuracy: 0.5475
Epoch 00001: val_accuracy improved from -inf to 0.64120, saving model to best.h5
20/20 [==============================] - 6s 310ms/step - loss: 0.6873 - accuracy: 0.5475 - val_loss: 0.6637 - val_accuracy: 0.6412
Epoch 2/50
20/20 [==============================] - ETA: 0s - loss: 0.6430 - accuracy: 0.6466
Epoch 00002: val_accuracy improved from 0.64120 to 0.72240, saving model to best.h5
20/20 [==============================] - 6s 280ms/step - loss: 0.6430 - accuracy: 0.6466 - val_loss: 0.6038 - val_accuracy: 0.7224
Epoch 3/50
20/20 [==============================] - ETA: 0s - loss: 0.5464 - accuracy: 0.7409
Epoch 00003: val_accuracy improved from 0.72240 to 0.80420, saving model to best.h5
20/20 [==============================] - 6s 283ms/step - loss: 0.5464 - accuracy: 0.7409 - val_loss: 0.4505 - val_accuracy: 0.8042
Epoch 4/50
20/20 [==============================] - ETA: 0s - loss: 0.4817 -

INFO:absl:Model trainning complete


Epoch 00027: early stopping


In [106]:
# validation & predict
model.load_weights('best.h5')
y_preds = model.predict(test_inputs)
logging.info("Prediction DONE")


sub = pd.read_csv('sample.csv')
sub['sentiment'] = y_preds
sub.to_csv('popcorn_submission.csv', index=False)
!curl -X PUT --upload-file popcorn_submission.csv ali.140714.xyz:8000/


INFO:absl:Prediction DONE


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 97  443k    0     0   97  432k      0  98764  0:00:04  0:00:04 --:--:-- 98742

## Received: "popcorn_submission.csv"

100  443k    0    41  100  443k      8  90540  0:00:05  0:00:05 --:--:-- 99580


In [0]:
sub = pd.read_csv('sample.csv')
sub['sentiment'] = y_preds
# sub.to_csv('popcorn_submission.csv', index=False)
sub

# Bert imbedding 

In [98]:
train, test = load_data()
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(train['text'], train['target'], random_state=0)

model = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1'
model = 'https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2'
hub_layer = hub.KerasLayer(model, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)

input = tf.keras.Input(shape=(), name="Input", dtype=tf.string)
net = hub_layer(input)
net = tf.keras.layers.Dense(64)(net)
net = tf.keras.layers.Dense(32)(net)
output = tf.keras.layers.Dense(1, activation='sigmoid')(net)
model = tf.keras.models.Model(input, output)
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None,)]                 0         
_________________________________________________________________
keras_layer_6 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_30 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_31 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 33        
Total params: 124,653,057
Trainable params: 124,653,057
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(
    loss = tf.losses.BinaryCrossentropy(from_logits=True),
    optimizer = 'adam',
    metrics = [tf.keras.metrics.BinaryAccuracy(name="accuracy")])

cp = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_weights.hdf5', monitor='val_accuracy', 
    verbose=1, save_best_only=True, mode='max')

# Early stopping 
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=False
)

In [100]:
history = model.fit(X_train, y_train,
          validation_data=(X_val, y_val),
          batch_size=128,
          callbacks=[cp, es],
          epochs = 30, verbose=1)

Epoch 1/30
147/147 [==============================] - ETA: 0s - loss: 0.6044 - accuracy: 0.7651
Epoch 00001: val_accuracy improved from -inf to 0.88608, saving model to best_weights.hdf5
147/147 [==============================] - 14s 92ms/step - loss: 0.6044 - accuracy: 0.7651 - val_loss: 0.5622 - val_accuracy: 0.8861
Epoch 2/30
147/147 [==============================] - ETA: 0s - loss: 0.5361 - accuracy: 0.9277
Epoch 00002: val_accuracy improved from 0.88608 to 0.88944, saving model to best_weights.hdf5
147/147 [==============================] - 40s 273ms/step - loss: 0.5361 - accuracy: 0.9277 - val_loss: 0.5585 - val_accuracy: 0.8894
Epoch 3/30
146/147 [============================>.] - ETA: 0s - loss: 0.5209 - accuracy: 0.9616
Epoch 00003: val_accuracy did not improve from 0.88944
147/147 [==============================] - 6s 42ms/step - loss: 0.5208 - accuracy: 0.9615 - val_loss: 0.5565 - val_accuracy: 0.8880
Epoch 4/30
147/147 [==============================] - ETA: 0s - loss: 0.5

In [101]:
model.load_weights('best_weights.hdf5')
y_preds = model.predict(test.text)
logging.info("Prediction DONE")


sub = pd.read_csv('sample.csv')
sub['sentiment'] = y_preds
sub.to_csv('popcorn_submission.csv', index=False)
!curl -X PUT --upload-file popcorn_submission.csv ali.140714.xyz:8000/

INFO:absl:Prediction DONE


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  453k    0     0  100  453k      0  91975  0:00:05  0:00:05 --:--:-- 92249

## Received: "popcorn_submission.csv"

100  453k    0    41  100  453k      8  91975  0:00:05  0:00:05 --:--:--  128k


In [93]:
y_preds

array([[1.0000000e+00],
       [9.0371032e-04],
       [5.4717118e-01],
       ...,
       [1.4462286e-03],
       [9.9999976e-01],
       [9.3779229e-02]], dtype=float32)